In [11]:
import typing as tp
from dataclasses import dataclass

import networkx as nx
import osmnx as ox
from shapely.geometry import LineString, Point
from tqdm import tqdm
import copy
import folium


class PathFinder:

    def __init__(
            self,
            start_point: tp.Tuple[float, float],
            end_point: tp.Tuple[float, float],
        ):
        self.graph: tp.Union[nx.Graph, nx.MultiDiGraph, None] = None
        self.start_point = start_point
        self.end_point = end_point

    def load_graph(self, radius_from_path: int = 50_000):
        """
        `radius_from_path` is given in meters.
        """
        line = LineString([self.start_point, self.end_point])
        buffered_polygon = line.buffer(radius_from_path)
        self.graph = ox.graph_from_polygon(buffered_polygon, network_type="all")
    

    def load_graph_from_region(self):
        region = 'Małopolskie Voivodeship, Poland'  # Adjust the region as needed
    
        # Download the street network for the region
        self.graph = ox.graph_from_place(region, network_type='all')

    def load_graph_from_radius(self, radius: int):
        self.graph = ox.graph_from_point(self.start_point, dist=radius, network_type='all')

    def save_graph(self):
        ox.save_graphml(self.graph, filepath="graph.graphml")

    def load_from_file(self, filepath: str = "graph.graphml"):
        self.graph = ox.load_graphml(filepath)

    def load_criteria(self):
        ATTRACTIVENESS = "attractiveness"
        for u, v, key, data in tqdm(self.graph.edges(keys=True, data=True)):
            
            # Adding attractiveness
            data[ATTRACTIVENESS] = data['length']

            # Weighting/excluding by speed
            if 'maxspeed' in data:
                for speed in list(data['maxspeed']):
                    try:
                        speed_value = int(speed)
                    except:
                        continue
                    if speed_value in range(0, 30):
                        penalty = 1
                    elif speed_value in range(30, 50):
                        penalty = 2
                    elif speed_value in range(50, 90):
                        penalty = 3
                    elif speed_value > 90:
                        penalty = 4
                    data[ATTRACTIVENESS] *= penalty

            # Weighting/excluding by type
            if 'highway' in data and data['highway'] == 'cycleway': # 'tertiary']:
                data[ATTRACTIVENESS] *= 2
            if 'highway' in data and data['highway'] == 'tertiary':
                data[ATTRACTIVENESS] *= 2

    def find_path(self):
        # Get nearest nodes to points A and B
        a_node = ox.distance.nearest_nodes(self.graph, self.start_point[1], self.start_point[0])
        b_node = ox.distance.nearest_nodes(self.graph, self.end_point[1], self.end_point[0])

        print("Finding shortest...")
        # Find the shortest path using custom weights
        shortest_path = nx.shortest_path(self.graph, a_node, b_node, weight='length')
        print("Found!")

        # Plot the graph and the shortest path
        # ox.plot_graph_route(self.graph, shortest_path)

        # Output path distance in meters
        distance = nx.shortest_path_length(self.graph, a_node, b_node, weight='length')
        print(f"Shortest path distance: {distance} meters")

        self.path = shortest_path

    def show_path(self):
        # Create a map centered on point A
        m = folium.Map(location=self.start_point, zoom_start=12)

        # Plot the shortest path based on edge geometries
        for u, v, key in zip(self.path[:-1], self.path[1:], range(len(self.path)-1)):
            # Retrieve edge data and check if it has a geometry (for curved roads)
            edge_data = self.graph.get_edge_data(u, v)
            print(edge_data)
            
            if 'geometry' in edge_data:
                # If edge has geometry (linestring), extract and plot the full geometry
                coords = list(edge_data['geometry'].coords)
            else:
                # If no geometry, plot a straight line between nodes
                coords = [(self.graph.nodes[u]['y'], self.graph.nodes[u]['x']),
                        (self.graph.nodes[v]['y'], self.graph.nodes[v]['x'])]
            
            # Add the segment to the map
            folium.PolyLine(coords, color="blue", weight=5, opacity=0.7).add_to(m)

        # Add markers for start (point A) and end (point B)
        folium.Marker(location=self.start_point, popup="Start: Point A", icon=folium.Icon(color="green")).add_to(m)
        folium.Marker(location=self.end_point, popup="End: Point B", icon=folium.Icon(color="red")).add_to(m)

        # Display the map
        return m

In [12]:
nowy_sacz = (49.639989612887234, 20.69236640147815)
nowy_targ: tuple[float, float] = (49.48271570006089, 20.0370067919616)

In [13]:
# pf = PathFinder(
#     nowy_sacz,
#     nowy_targ,
# )

In [15]:
pf2 = PathFinder(
    nowy_sacz,
    nowy_targ,
)
pf2.graph = pf.graph
pf = pf2

In [5]:
# pf.load_graph_from_region()

In [6]:
# pf.save_graph()

In [7]:
# pf.load_from_file()

In [16]:
pf.load_criteria()

 19%|█▉        | 340162/1757555 [00:00<00:02, 544243.03it/s]

P
L
:
r
u
r
a
l
P
L
:
u
r
b
a
n
P
L
:
u
r
b
a
n
P
L
:
u
r
b
a
n


 35%|███▍      | 613160/1757555 [00:01<00:02, 485589.78it/s]

P
L
:
r
u
r
a
l
PL:urban
PL:rural
P
L
:
u
r
b
a
n
PL:urban
PL:rural


 59%|█████▉    | 1042410/1757555 [00:02<00:01, 504219.68it/s]

P
L
:
u
r
b
a
n
PL:urban
P
L
:
u
r
b
a
n
P
L
:
u
r
b
a
n
P
L
:
u
r
b
a
n
PL:urban


 82%|████████▏ | 1445549/1757555 [00:02<00:00, 472602.70it/s]

PL:rural
PL:rural


100%|██████████| 1757555/1757555 [00:03<00:00, 497009.10it/s]


In [17]:
pf.find_path()

Finding shortest...
Found!
Shortest path distance: 41521.73200000002 meters


In [18]:
pf.show_path()

{0: {'osmid': 75084154, 'name': 'Powstańców Śląskich', 'highway': 'residential', 'oneway': False, 'reversed': False, 'length': 123.047, 'attractiveness': 123.047}}
{0: {'osmid': 75084183, 'name': 'Powstańców Wielkopolskich', 'highway': 'residential', 'oneway': False, 'reversed': False, 'length': 47.278, 'geometry': <LINESTRING (20.692 49.639, 20.692 49.639, 20.692 49.639)>, 'attractiveness': 47.278}}
{0: {'osmid': 75084183, 'name': 'Powstańców Wielkopolskich', 'highway': 'residential', 'oneway': False, 'reversed': False, 'length': 77.222, 'geometry': <LINESTRING (20.692 49.639, 20.691 49.639, 20.69 49.639)>, 'attractiveness': 77.222}}
{0: {'osmid': 353598389, 'oneway': False, 'name': 'Warzywna', 'highway': 'residential', 'reversed': False, 'length': 11.264, 'attractiveness': 11.264}}
{0: {'osmid': 353598389, 'oneway': False, 'name': 'Warzywna', 'highway': 'residential', 'reversed': False, 'length': 36.439, 'attractiveness': 36.439}}
{0: {'osmid': 353598389, 'oneway': False, 'name': 'Wa